In [ ]:
import pandas as pd

#load data return table and interpret correctly
df = pd.read_csv('surgery_procedure.csv', encoding='utf-8', na_values=[''])

#check the required columns exist
always_required = [
    'path_no',
    'surgery_date',
    'surgery_proc',
    'surgery_side',
    'surgery_sample_side',
    'lesion_focality',
    'surgery_invasive',
    'surgery_insitu',
    'receptor_er_status',
    'receptor_pr_status',
    'receptor_her_status'
]
for col in always_required:
    if col not in df.columns:
        raise ValueError(f"Missing required column: {col}")

# Also check that the conditional column exists
if 'surgery_insitu_type' not in df.columns:
    raise ValueError("Missing required column: surgery_insitu_type")

# 3. Coerce surgery_insitu to numeric so both '1' and 1 become 1.0
insitu_num = pd.to_numeric(df['surgery_insitu'], errors='coerce')

# 4. Build mask:
#    (a) any of the always_required columns is NaN
#    (b) OR surgery_insitu == 1 but surgery_insitu_type is NaN
missing_always = df[always_required].isna().any(axis=1)
missing_insitu_type = (insitu_num == 1) & df['surgery_insitu_type'].isna()
mask_missing = missing_always | missing_insitu_type

# 5. Filter and export bad records
df_missing = df.loc[mask_missing].copy()
df_missing.to_csv('surgery_procedure_missing.csv', index=False)

print(f"{len(df_missing)} records found missing required surgery fields.")
